In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from pprint import pprint
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import os

In [2]:
LABEL_INT_DICT = np.unique(pd.read_csv('D:/data_ai/5_AI/ucf11_train_vid.csv')['label'])
pprint(LABEL_INT_DICT)
LABEL_STR_DICT = {k:v for v, k in enumerate(LABEL_INT_DICT)}    # KEY값과 VALUE값 바꿈
pprint(LABEL_STR_DICT)

array(['biking', 'diving', 'golf', 'juggle', 'jumping', 'riding',
       'shooting', 'spiking', 'swing', 'tennis', 'walk'], dtype=object)
{'biking': 0,
 'diving': 1,
 'golf': 2,
 'juggle': 3,
 'jumping': 4,
 'riding': 5,
 'shooting': 6,
 'spiking': 7,
 'swing': 8,
 'tennis': 9,
 'walk': 10}


In [3]:
model = keras.models.load_model(
    'D:/data_ai/5_AI/ucf11_weights/05-0.96.hdf5',
    custom_objects={'KerasLayer':hub.KerasLayer}
)

model.build([None, 256, 256, 3])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              5919312   
                                                                 
 dense (Dense)               (None, 11)                14091     
                                                                 
Total params: 5,933,403
Trainable params: 5,872,795
Non-trainable params: 60,608
_________________________________________________________________


In [4]:
csv_path = 'D:/data_ai/5_AI/ucf11_valid_vid.csv'
valid_df = pd.read_csv(csv_path)

idx = 21
elem = valid_df.iloc[idx]

cap = cv2.VideoCapture(
    elem['file_path']
)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter('D:/data_ai/5_AI/movie.avi', fourcc, 30, (256, 256), True)

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ = frame_ / 255
    
    # np.newaxis : 1 차원 추가 => 리스트 하나 추가[[리스트],[원래 배열]] => 결국 1차원, ... : 그대로 파라미터 채워주는 것
    # 비교하기 위해
    pred = model.predict(frame_[np.newaxis, ...])    
    label = LABEL_INT_DICT[np.argmax(pred)]
    
    frame = cv2.resize(frame, (256, 256))
    cv2.putText(frame, label, (50, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 5)
    writer.write(frame)
    
cap.release()
writer.release()

# 예측 값이 튀면 결과 값이 변할 수 있음

1/1 [==============================] - 0s 84ms/step


1/1 [==============================] - 0s 72ms/step


In [6]:
cap = cv2.VideoCapture(
    elem['file_path']
)

fourcc = cv2.VideoWriter_fourcc(*'MJPG')
writer = cv2.VideoWriter('D:/data_ai/5_AI/roll.avi', fourcc, 30, (256, 256), True)

queue = []

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame_ = frame.copy()
    frame_ = cv2.cvtColor(frame_, cv2.COLOR_BGR2RGB)
    frame_ = cv2.resize(frame_, (256, 256))
    frame_ = frame_.astype('float32')
    frame_ = frame_ / 255
    
    print(frame_)
    print("*" * 30)
    # np.newaxis : 1 차원 추가 => 리스트 하나 추가[[리스트],[원래 배열]] => 결국 1차원, ... : 그대로 파라미터 채워주는 것
    # 비교하기 위해
    pred = model.predict(frame_[np.newaxis, ...])   
    print(pred)
    
    queue.append(pred)
    
    # 평균으로 하여 결과 값을 안정적으로 리턴
    results = np.array(queue).mean(axis=0)
    print("✔" * 30)
    print(results)
    print("✔" * 30)
    label = LABEL_INT_DICT[np.argmax(results)]
    
    frame = cv2.resize(frame, (256, 256))
    cv2.putText(frame, label, (50, 45), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 5)
    writer.write(frame)
    
cap.release()
writer.release()

[[[0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 ...

 [[0.3529412  0.34509805 0.32941177]
  [0.3529412  0.34509805 0.32941177]
  [0.3529412  0.34509805 0.32941177]
  ...
  [0.30588236 0.29803923 0.28235295]
  [0.30588236 0.29803923 0.28235295]
  [0.30588236 0.29803923 0.28235295]]

 [[0.3529412  0.34509805 0.32941177]
  [0.3529412  0.34509805 0.32941177]


1/1 [==============================] - 0s 67ms/step
[[1.8871742e-06 1.0900598e-06 1.1608872e-06 1.3571655e-06 2.0168674e-05
  9.9996042e-01 1.0528302e-07 7.5752826e-07 2.7388498e-06 6.5969658e-07
  9.6259146e-06]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[5.9711101e-06 2.1865892e-06 1.9064807e-06 3.3532415e-06 3.3783952e-05
  9.9991095e-01 3.1523584e-07 3.3617439e-06 4.2264246e-06 1.2036901e-06
  3.2720931e-05]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.9

1/1 [==============================] - 0s 72ms/step
[[1.16805466e-04 7.62718309e-06 6.14855389e-06 1.80977577e-06
  2.99636886e-04 9.99370396e-01 8.98775966e-07 3.27683392e-06
  1.91950639e-05 6.26384644e-06 1.67905455e-04]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[6.5172542e-05 6.3030989e-06 4.5053675e-06 3.3374376e-06 1.7749198e-04
  9.9956447e-01 6.0823669e-07 3.3236288e-06 2.0268939e-05 3.4365992e-06
  1.5113616e-04]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.5294118  0.58431375 0.6313726 ]
  [0.46666667 0.52156866 0.5686275 ]
  [0.41568628 0.46666667 0.5176471 ]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.45490196 0.5137255  0.56078434]
  [0.40392157 0.4627451  0.5058824 ]
  [0.3647059  0.41960785 0.46666667]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.42352942 0.47843137 0.5254902 ]
  [0.39607844 0.4509804  0.49803922]
  [0.3764706  0.4

1/1 [==============================] - 0s 81ms/step
[[1.1138477e-06 9.4029855e-07 4.8223285e-07 1.2208900e-06 5.0751601e-06
  9.9998391e-01 1.4801375e-07 9.7581824e-07 1.4823618e-06 6.7299703e-07
  3.9882752e-06]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[4.5916615e-05 4.6320879e-06 3.2609953e-06 2.4770056e-06 1.2382781e-04
  9.9969679e-01 4.5596875e-07 2.4154288e-06 1.4731004e-05 2.6095629e-06
  1.0284736e-04]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.9843137  1.         1.        ]
  [0.9843137  1.         1.        ]
  [0.9843137  1.         1.        ]
  ...
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]]

 [[0.9843137  1.         1.        ]
  [0.9843137  1.         1.        ]
  [0.9843137  1.         1.        ]
  ...
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]]

 [[0.9843137  1.         1.        ]
  [0.9843137  1.         1.        ]
  [0.9843137  1.         1. 

1/1 [==============================] - 0s 89ms/step
[[6.3351454e-06 2.3697353e-06 1.0626105e-06 1.8917101e-06 3.1513293e-05
  9.9993873e-01 2.8994992e-07 1.8334698e-06 7.0827359e-06 1.0986095e-06
  7.7922205e-06]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[3.5201465e-05 3.9206598e-06 2.7173844e-06 2.2650722e-06 9.9116805e-05
  9.9975955e-01 4.0544555e-07 2.2769455e-06 1.2219777e-05 2.2256818e-06
  8.0023718e-05]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.9529412  0.95686275 0.96862745]
  [0.9529412  0.95686275 0.96862745]
  [0.9529412  0.95686275 0.96862745]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.95686275 0.9607843  0.972549  ]
  [0.95686275 0.9607843  0.972549  ]
  [0.95686275 0.9607843  0.972549  ]
  ...
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]
  [0.92941177 0.91764706 0.93333334]]

 [[0.9529412  0.95686275 0.96862745]
  [0.9529412  0.95686275 0.96862745]
  [0.9529412  0.95686275 0.9

...

1/1 [==============================] - 0s 97ms/step
[[1.7806508e-06 5.7879993e-07 1.0706170e-07 1.0187379e-06 1.4434432e-06
  9.9999189e-01 1.7762099e-07 3.3694423e-07 1.5414968e-06 9.7752763e-07
  2.3432288e-07]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[2.8713778e-04 1.3601973e-05 5.7946331e-06 3.9526071e-06 2.0564713e-03
  9.9653482e-01 1.7624000e-06 6.8747227e-06 2.6534146e-05 2.8238185e-06
  1.0600403e-03]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.47843137 0.4862745  0.54901963]
  [0.5019608  0.50980395 0.57254905]
  [0.5294118  0.5372549  0.6       ]
  ...
  [0.7764706  0.7921569  0.8509804 ]
  [0.83137256 0.84313726 0.90588236]
  [0.8627451  0.87058824 0.93333334]]

 [[0.47843137 0.4862745  0.54901963]
  [0.5019608  0.50980395 0.57254905]
  [0.5294118  0.5372549  0.6       ]
  ...
  [0.7647059  0.78039217 0.8392157 ]
  [0.8235294  0.83137256 0.89411765]
  [0.8509804  0.85882354 0.92156863]]

 [[0.47843137 0.4862745  0.54901963]
  [0.5019608  0.50980395 0.57254905]
  [0.5294118  0.5372549  0.6

1/1 [==============================] - 0s 124ms/step
[[1.8655439e-06 1.3699849e-06 8.0966863e-08 9.9715680e-07 4.0888535e-06
  9.9998820e-01 1.6986158e-07 3.5997579e-07 1.0889886e-06 1.3851692e-06
  5.3531676e-07]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[2.8079754e-04 1.3316919e-05 5.6675399e-06 3.8828175e-06 2.0108160e-03
  9.9661160e-01 1.7265762e-06 6.7287583e-06 2.5971118e-05 2.7820872e-06
  1.0364901e-03]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.3764706  0.39607844 0.37254903]
  [0.3764706  0.39607844 0.37254903]
  [0.3764706  0.39607844 0.37254903]
  ...
  [0.53333336 0.54509807 0.5803922 ]
  [0.49803922 0.5137255  0.54901963]
  [0.4745098  0.4862745  0.52156866]]

 [[0.3764706  0.39607844 0.37254903]
  [0.3764706  0.39607844 0.37254903]
  [0.3764706  0.39607844 0.37254903]
  ...
  [0.53333336 0.54509807 0.5803922 ]
  [0.49803922 0.5058824  0.54509807]
  [0.4862745  0.49803922 0.53333336]]

 [[0.3764706  0.39607844 0.37254903]
  [0.3764706  0.39607844 0.37254903]
  [0.3764706  0.39607844 0.

1/1 [==============================] - 0s 73ms/step
[[1.7167021e-06 7.3529827e-07 1.4983176e-07 1.2567120e-05 3.1470488e-06
  9.9997139e-01 3.1888874e-07 7.3458614e-07 1.4981455e-06 2.4051330e-06
  5.2693549e-06]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[2.7472700e-04 1.3054943e-05 5.5461182e-06 3.9415922e-06 1.9671614e-03
  9.9668479e-01 1.6958594e-06 6.5922763e-06 2.5435407e-05 2.7680830e-06
  1.0140014e-03]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.70980394 0.7607843  0.827451  ]
  [0.6784314  0.73333335 0.8       ]
  [0.627451   0.6784314  0.74509805]
  ...
  [0.5686275  0.5882353  0.6901961 ]
  [0.5176471  0.5372549  0.6392157 ]
  [0.4862745  0.50980395 0.6117647 ]]

 [[0.70980394 0.7607843  0.827451  ]
  [0.68235296 0.7294118  0.8       ]
  [0.627451   0.6784314  0.74509805]
  ...
  [0.5686275  0.5882353  0.6901961 ]
  [0.5176471  0.5372549  0.6392157 ]
  [0.4862745  0.5058824  0.6117647 ]]

 [[0.70980394 0.7607843  0.827451  ]
  [0.6784314  0.7294118  0.8       ]
  [0.627451   0.6784314  0.7

1/1 [==============================] - 0s 109ms/step
[[3.8914245e-06 1.2020654e-06 5.8066991e-08 4.2074607e-06 1.9827858e-06
  9.9998319e-01 4.0557268e-07 4.9748849e-07 1.4399233e-06 2.9849252e-06
  1.9996035e-07]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[2.68992648e-04 1.28029715e-05 5.42974294e-06 3.93391838e-06
  1.92535599e-03 9.96755004e-01 1.67088524e-06 6.46282979e-06
  2.49283858e-05 2.76391870e-06 9.92439920e-04]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[[0.7294118  0.7529412  0.827451  ]
  [0.7294118  0.7529412  0.827451  ]
  [0.7294118  0.7529412  0.827451  ]
  ...
  [0.6627451  0.6745098  0.7529412 ]
  [0.6627451  0.6745098  0.75686276]
  [0.6627451  0.6745098  0.7607843 ]]

 [[0.7294118  0.7529412  0.827451  ]
  [0.7294118  0.7529412  0.827451  ]
  [0.7294118  0.7529412  0.827451  ]
  ...
  [0.6627451  0.6745098  0.7529412 ]
  [0.6627451  0.6745098  0.75686276]
  [0.6627451  0.6745098  0.7607843 ]]

 [[0.7294118  0.7529412  0.827451  ]
  [0.7294118  0.7529412  0.827451  ]
  [0.7294118  0.

1/1 [==============================] - 0s 83ms/step
[[4.4320404e-06 2.4280694e-06 1.0684451e-07 5.8828273e-06 2.0235898e-06
  9.9998105e-01 2.8922616e-07 5.8102904e-07 1.0623069e-06 1.6057772e-06
  6.3857448e-07]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
[[2.6366301e-04 1.2577010e-05 5.3194499e-06 3.9462593e-06 1.8852997e-03
  9.9682206e-01 1.6430772e-06 6.3444754e-06 2.4448185e-05 2.7485105e-06
  9.7177963e-04]]
✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔✔
